In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
from glob import glob
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications.vgg16 import VGG16 as PretrainedModel, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_path = 'data/train_set'
test_path = 'data/test_set'

In [ ]:
IMAGE_SIZE = [200, 200]

In [ ]:
ptm = PretrainedModel(input_shape= IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
x = Flatten()(ptm.output)

In [ ]:
model = Model(inputs=ptm.input, outputs=x)

In [ ]:
model.summary()

In [ ]:
gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
batch_size = 128

train_generator = gen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size= batch_size,
    class_mode='binary'
)

test_generator = gen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size= batch_size,
    class_mode='binary'
)

In [ ]:
NTrain = 3000
NTest = 1000

# Detect shape for the flatten output
feat = model.predict(np.random.random([1] + IMAGE_SIZE + [3]))
D = feat.shape[1]

X_train = np.zeros((NTrain, D))
Y_train = np.zeros(NTrain)
X_test = np.zeros((NTest, D))
Y_test = np.zeros(NTest)

In [ ]:
# populate X_train and Y_train
i = 0
for x,y in train_generator:
    features = model.predict(x)
    
    #batch size if applicable
    sz = len(y)
    X_train[i:i+sz] = features
    Y_train[i:i+sz] = y
    
    i = i+sz
    
    if i>=NTrain:
        break
    
    
print(i)    

In [ ]:
# populate X_train and Y_train
i = 0
for x,y in test_generator:
    features = model.predict(x)
    
    #batch size if applicable
    sz = len(y)
    X_test[i:i+sz] = features
    Y_test[i:i+sz] = y
    
    i = i+sz
    
    if i>=NTest:
        break
    
    
print(i)   

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_transform(X_train)
sc.transform(Y_train)

In [ ]:
#Using SKlearn
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression()
logr.fit(X_train,Y_train)
print(logr.score(X_train,Y_train))
print(logr.score(X_test,Y_test))

In [ ]:
#Using Tensorflow
i = Input(shape=(D,))
x = Dense(1, activation='sigmoid')(i)
lm = Model(i,x)

In [ ]:
lm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
r = lm.fit(X_train, Y_train, batch_size=128, epochs=10, validatio_data=(X_test, Y_test))

In [ ]:
# Plot loss graph
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
# Plot accuracy graph
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()